<img src='figs/CapricornCurrent.gif' />

# eReefs hydrodynamics specific GBR area

In [ ]:
import os
import numpy as np

import datetime as dt

import netCDF4
from netCDF4 import Dataset, num2date

from pylab import *
import matplotlib.ticker as mticker
import matplotlib.tri as triangle 
from matplotlib import pyplot as plt, animation

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import cmocean

import cartopy
import cartopy.crs as ccrs
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))

from IPython.display import HTML, display

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

## Define which data to be plotted.

In this section we define which data we want to read and plot.

- **inputFile**
  The netCDF input file. This can either be a downloaded file (see [How to manually download derived data from THREDDS](http://ereefs.aims.local/ereefs-aims/help/how-to-manually-download-derived-data)) or a  OPeNDAP URL from the [AIMS THREDDS server](http://thredds.ereefs.aims.gov.au/thredds/catalog.html). For this tutorial we are using the OPeNDAP URL.
- **selectedVariable**
  The name of the variable in the netCDF file.
- **selectedTimeIndex**
  The time slice in the netCDF file. Note the index starts with 0. For example, in the netCDF file, the time steps are "days". This means if you select `selectedTimeIndex=1` it refers to the second day in the netCDF file.
- **selectedDepthIndex**
  The depth slice in the netCDF file. Note the index starts with 0. 

# Connect to the OpeNDAP endpoint for a specified month. 

We query the server based on the OPeNDAP URL for a specific file "EREEFS_AIMS-CSIRO_gbr4_v2_hydro_daily-monthly-YYYY-MM.nc". 


- **gbr4**: we use the Hydrodynamic 4km model and daily data for the month specified

In [ ]:
month = 3
year = 2020

netCDF_datestr = str(year)+'-'+format(month, '02')
print('File chosen time interval:',netCDF_datestr)

inputFile = "http://thredds.ereefs.aims.gov.au/thredds/dodsC/s3://aims-ereefs-public-prod/derived/ncaggregate/ereefs/gbr4_v2/daily-monthly/EREEFS_AIMS-CSIRO_gbr4_v2_hydro_daily-monthly-"+netCDF_datestr+".nc"

We now load the dataset within the Jupyter environment...

In [ ]:
nc_data = Dataset(inputFile, 'r')

print('Get the list of variable in the file:')
print(list(nc_data.variables.keys()))

ncdata = nc_data.variables

To get information for a specific variables, we can use the following:

In [ ]:
ncdata['mean_cur']

In [ ]:
ncdata['mean_cur'].standard_name

In [ ]:
ncdata['mean_cur'].units

# Load variables

We then load the file dataset in Jupyter

In [ ]:
# Starting with the spatial domain
lat = ncdata['latitude'][:].filled(fill_value=0.)
lon = ncdata['longitude'][:].filled(fill_value=0.)

print('eReefs model spatial extent:\n')
print(' - Longitudinal extent:',lon.min(),lon.max())
print(' - Latitudinal extent:',lat.min(),lat.max())

In [ ]:
# Get time span of the dataset
time_var = ncdata['time']

# Starting time
dtime = netCDF4.num2date(time_var[0],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M')
print(' - start time: ',daystr,'\n')

# Ending time
dtime = netCDF4.num2date(time_var[-1],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M')
print(' - end time: ',daystr,'\n')

ntime = len(time_var)

print(' - Number of time steps',ntime,'\n')

In [ ]:
# Number of vetical points along the z-coordinate model
zc = ncdata['zc'][:].filled(fill_value=0.)
nlay = len(zc)

print('Number of vertical layers',nlay)

for k in range(nlay):
    print(f'  + vertical layer {k} is at {zc[k]} m')

# Meshing dataset for visualisation

To be able to visualise the variables on a map we first have to use the longitude and latidude values to mesh. 

Here we will build a triangular grid based on the Dataset coordinates (using the *triangle.Triangulation* from the **Matplotlib** library).

In [ ]:
if len(lat.flatten()) == len(lon.flatten()):
    meshtri = triangle.Triangulation(lon.flatten(),lat.flatten())
else:
    loni, lati = np.meshgrid(lon, lat)
    lonlati = np.dstack([loni.flatten(), lati.flatten()])[0]
    meshtri = triangle.Triangulation(loni.flatten(),lati.flatten())

In this notebook, we will evaluate hydrodynamic conditions on a specific area. Here we focus on the **Capricorn Bunker Group** (Southern GBR, around One Tree Reef). The region extent is set with:

- lon min/max:  150.4/152.6
- lat min/max:  -24.1/-23.1

In [ ]:
latExtent = [-24.1,-23.1]
lonExtent = [150.4,152.6]

Now we will use the function **eReefs_Area_Model** below to plot the different variables at specific time step and depth for the region of interest.

In [ ]:
def find_nearest(array, value):
    '''
    Find index of nearest value in a numpy array
    '''
    
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    
    return idx


def eReefs_Area_Model(tstep, depth, lonExtent, latExtent, dataname, 
                      datalvl, color, size, fname, vecsample, vecscale,
                      veclenght, show=False, vecPlot=False):
    
    '''
    This function plots for a specified time index and depth the value of a variable within a defined region based on the 
    eReefs netCDF file.
    
    args:
    - tstep: specified time index
    - depth: specified depth layer
    - lonExtent: region longitudinal range specified as a list [min,max] 
    - latExtent: region latitudinal range specified as a list [min,max] 
    - dataname: specified variable name 
    - datalvl: range of the variable values specified as a list [min,max] 
    - color: colormap to use for the plot (here one can use the cmocean library: https://matplotlib.org/cmocean/#installation)
    - size: figure size  
    - fname: figure name when saved on disk, it is worth noting that the specified time index and depth layer will be automatically added
    - vecsample: sampling on velocity arrows to plot on the maps when velocity verctor are used
    - vecscale: scaling velocity arrows on the maps when velocity verctor are used
    - veclenght: lenght of the reference vector (in m/s)
    - show: set to True when the map is shown in the jupyter environment directly 
    - vecPlot: set to True when the current flow vector are plotted
    '''
    
    cmap = color
    cmap.set_over('0')
    cmap.set_under(cmap(0))

    lonIDmin = find_nearest(lon, lonExtent[0])
    lonIDmax = find_nearest(lon, lonExtent[1])

    latIDmin = find_nearest(lat, latExtent[0])
    latIDmax = find_nearest(lat, latExtent[1])

    # Get data
    data = ncdata[dataname][tstep, depth, :,:]
    data[np.isnan(data)] = 1000
    levels = np.arange(datalvl[0],datalvl[1],0.05)
    
    fig = plt.figure(figsize=size, facecolor='w', edgecolor='k')

    ax = plt.axes(projection=ccrs.Mercator()) #projection=ccrs.PlateCarree())
    ax.set_extent([lonExtent[0], lonExtent[1], latExtent[1], latExtent[0]], ccrs.PlateCarree())

    cf = plt.tricontourf(meshtri, data.flatten(), 
                transform=ccrs.PlateCarree(), 
                levels=levels,
                cmap=cmap,
                extend='min')

    # Plot velocity arrows 
    if vecPlot:
        u = ncdata['u'][tstep, depth, :,:].filled(fill_value=0.)
        v = ncdata['v'][tstep, depth, :,:].filled(fill_value=0.)
        # find non zeros velocity points
        ind = np.where(np.logical_and(data.flatten()>0,data.flatten()<1000))[0]
        np.random.shuffle(ind)
        Nvec = int(len(ind) / vecsample)
        idv = ind[:Nvec]
        Q = plt.quiver(loni.flatten()[idv],
                       lati.flatten()[idv],
                       u.flatten()[idv],
                       v.flatten()[idv],
                       transform=ccrs.PlateCarree(), 
                       scale=vecscale)

        maxstr='%3.2f m/s' % veclenght
        qk = plt.quiverkey(Q,1.06,0.,veclenght,maxstr,labelpos='S')


    # Color bar
    gca().patch.set_facecolor('1')
    cbar = fig.colorbar(cf, ax=ax, fraction=0.02, pad=0.03)
    cbar.set_label(ncdata[dataname].units, 
                   rotation=-90, labelpad=15, fontsize=10)
    cbar.ax.tick_params(labelsize=8)

    # Title
    dtime = netCDF4.num2date(time_var[tstep],time_var.units)
    daystr = dtime.strftime('%Y-%b-%d %H:%M')
    plt.title(ncdata[dataname].short_name+', %s UTC+10' % (daystr), fontsize=11);

    # Plot lat/lon grid 
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=0.1, color='k', alpha=1, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 8}
    gl.ylabel_style = {'size': 8} 
    
    # Add map features
    ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '10m', 
                                                edgecolor='face', 
                                                facecolor='lightgray'))
    ax.coastlines(linewidth=1)

    if show:
        plt.tight_layout()
        plt.show()
    else:   
        plt.savefig(f"{fname}_time{tstep:04}_zc{depth:04}.png",dpi=300, 
                    bbox_inches='tight')

    fig.clear()
    plt.close(fig)
    plt.clf()
    
    return

Let's use the function. First we will check for a specific depth (here we chose the surface) each variable range:

In [ ]:
selectedDepthIndex = -1 

lonIDmin = find_nearest(lon, lonExtent[0])
lonIDmax = find_nearest(lon, lonExtent[1])

latIDmin = find_nearest(lat, latExtent[0])
latIDmax = find_nearest(lat, latExtent[1])

print('Dataset array indices for longitude and latitude containing the region:')
print('  - indices for longitude: ',lonIDmin,lonIDmax)
print('  - indices for latitude:  ',latIDmin,latIDmax)

print(' ')
print('Mean current range: ')
print(np.nanmin(ncdata['mean_cur'][:, selectedDepthIndex, lonIDmin:lonIDmax, latIDmin:latIDmax]),
      np.nanmax(ncdata['mean_cur'][:, selectedDepthIndex, lonIDmin:lonIDmax, latIDmin:latIDmax]))

print(' ')
print('Temperature range: ')
print(np.nanmin(ncdata['temp'][:, selectedDepthIndex, lonIDmin:lonIDmax, latIDmin:latIDmax]),
      np.nanmax(ncdata['temp'][:, selectedDepthIndex, lonIDmin:lonIDmax, latIDmin:latIDmax]))

print(' ')
print('Salinity range: ')
print(np.nanmin(ncdata['salt'][:, selectedDepthIndex, lonIDmin:lonIDmax, latIDmin:latIDmax]),
      np.nanmax(ncdata['salt'][:, selectedDepthIndex, lonIDmin:lonIDmax, latIDmin:latIDmax]))

We will plot the mean current like this:

In [ ]:
selectedTimeIndex = 1 
selectedVariable = 'mean_cur' 

# Vector field mapping information
veclenght = 0.5
vecscale = 20
vecsample = 5

# Figure size
size = (9, 8)

# Used color
color = cmocean.cm.speed

# Variable range for the colorscale
curlvl = [0.01,1.5]

# Saved file name
fname = 'CapricornCurrent'

# We now call the funct
eReefs_Area_Model(selectedTimeIndex, selectedDepthIndex, lonExtent, latExtent,
                  selectedVariable, curlvl, color, size, fname, vecsample, vecscale,
                  veclenght, show=True, vecPlot=True)

We can use a loop to plot changes over time like this:

In [ ]:
# Here we loop over the days with a 2 day increment
for tstep in range(0,ntime,2): 
    
    # Get time interval
    dtime = netCDF4.num2date(time_var[tstep],time_var.units)
    daystr = dtime.strftime('%Y-%b-%d %H:%M')
    print('Processing map at time: ',daystr)
    
    # Run the plotting function
    eReefs_Area_Model(tstep, selectedDepthIndex, lonExtent, latExtent,
                  selectedVariable, curlvl, color, size, fname, vecsample, vecscale,
                  veclenght, show=False, vecPlot=True)

We can then use the following line to create an animated `gif` 

In [ ]:
!convert -delay 100 CapricornCurrent_time*_zc-001.png CapricornCurrent.gif

It can then be displayed as:

In [ ]:
display(HTML("<img src='CapricornCurrent.gif' />"))

Same can be done to analyse salinity and temperature

### Temperature

In [ ]:
selectedTimeIndex = 1 
selectedVariable = 'temp' 

# Figure size
size = (9, 8)

# Used color
color = cmocean.cm.thermal

# Variable range for the colorscale
templvl = [20,31]

# Saved file name
fname = 'CapricornTemp'

# We now call the funct
eReefs_Area_Model(selectedTimeIndex, selectedDepthIndex, lonExtent, latExtent,
                  selectedVariable, templvl, color, size, fname, vecsample, vecscale,
                  veclenght, show=True, vecPlot=False)

In [ ]:
# Here we loop over the days with a 2 day increment
for tstep in range(0,ntime,2): 
    
    # Get time interval
    dtime = netCDF4.num2date(time_var[tstep],time_var.units)
    daystr = dtime.strftime('%Y-%b-%d %H:%M')
    print('Processing map at time: ',daystr)
    
    # Run the plotting function
    eReefs_Area_Model(tstep, selectedDepthIndex, lonExtent, latExtent,
                  selectedVariable, templvl, color, size, fname, vecsample, vecscale,
                  veclenght, show=False, vecPlot=False)

In [ ]:
!convert -delay 100 CapricornTemp_time*_zc-001.png CapricornTemp.gif

In [ ]:
display(HTML("<img src='CapricornTemp.gif' />"))

### Salinity

In [ ]:
selectedTimeIndex = 1 
selectedVariable = 'salt' 

# Figure size
size = (9, 8)

# Used color
color = cmocean.cm.haline

# Variable range for the colorscale
saltlvl = [20,37]

# Saved file name
fname = 'CapricornSalt'

# We now call the funct
eReefs_Area_Model(selectedTimeIndex, selectedDepthIndex, lonExtent, latExtent,
                  selectedVariable, saltlvl, color, size, fname, vecsample, vecscale,
                  veclenght, show=True, vecPlot=False)

In [ ]:
# Here we loop over the days with a 2 day increment
for tstep in range(0,ntime,2): 
    
    # Get time interval
    dtime = netCDF4.num2date(time_var[tstep],time_var.units)
    daystr = dtime.strftime('%Y-%b-%d %H:%M')
    print('Processing map at time: ',daystr)
    
    # Run the plotting function
    eReefs_Area_Model(tstep, selectedDepthIndex, lonExtent, latExtent,
                  selectedVariable, saltlvl, color, size, fname, vecsample, vecscale,
                  veclenght, show=False, vecPlot=False)

In [ ]:
!convert -delay 100 CapricornSalt_time*_zc-001.png CapricornSalt.gif

In [ ]:
display(HTML("<img src='CapricornSalt.gif' />"))